In [ ]:
import importlib
import layers.tied
importlib.reload(layers)
import tensorflow as tf
#tf.enable_eager_execution()
print(tf.__version__)

In [ ]:
def eval_tensor(t):
    return tf.Session().run(t)

def tensor2const(t):
    return tf.constant(eval_tensor(t))


Rx = tensor2const(tf.random.uniform([1,100]))
Ry = tensor2const(tf.random.uniform([1,50]))
TDxy = layers.tied.TiedDenseLayer(units=50) 
TDxy.build(input_shape=(100,))  # must build TDxy if we want to tie TDyx to it - we need the kernel
TDyx = layers.tied.TiedDenseLayer(units=100, tied_kernel=tf.transpose(TDxy.kernel))



In [ ]:
#with tf.name_scope("tied") as scope:
#X = tf.placeholder(tf.float32, shape=(None, 100))
#Y = tf.placeholder(tf.float32, shape=(None, 50))

assert(not tf.executing_eagerly())
inputs = tf.keras.Input(shape=(100,))  ##tensor=X)
outputs = TDxy(inputs)
Xreconstruct = TDyx(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
#model = tf.keras.models.Sequential([TDxy])
def loss_func(y0, y1):
    return (tf.losses.mean_squared_error(y0, y1) *0.5+ 
            0.5 * tf.losses.mean_squared_error(inputs, Xreconstruct))
    
optimizer = tf.train.MomentumOptimizer(use_nesterov=True, learning_rate=0.001, momentum=0.8)
##train_step = optimizer.minimize(loss)
model.compile(optimizer, loss=loss_func) ##'mean_squared_error')

model.fit(Rx, Ry, epochs=50,steps_per_epoch=1)

In [ ]:
W = TDxy.get_weights()[0]
#print(W.shape)
print(Rx[0].shape)
print (tf.matmul(Rx , W ).shape)
print("way1 loss:", eval_tensor(tf.nn.l2_loss(tf.matmul(Rx , W ) - Ry)/50))

Wrev = TDyx.get_weights()[0]
print(Wrev.shape)
print("way2 loss:", eval_tensor(tf.nn.l2_loss(tf.matmul(Ry , Wrev ) - Rx)/100))


In [ ]:
W = TDxy.get_weights()[0]
#print(W.shape)
print(Rx[0].shape)
print (tf.matmul(Rx , W ).shape)
print("way1 loss:", eval_tensor(tf.nn.l2_loss(tf.matmul(Rx , W ) - Ry)/50))

Wrev = TDyx.get_weights()[0]
print(Wrev.shape)
print("way2 loss:", eval_tensor(tf.nn.l2_loss(tf.matmul(Ry , Wrev ) - Rx)/100))


In [ ]:
TDyx.kernel.shape

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

onehot = lambda y: tf.one_hot(y, depth=10)
flat = lambda x : tf.reshape(x, (-1, x.shape[1] * x.shape[2]))
(x_train, y_train_labels),(x_test, y_test_labels) = mnist.load_data()
y_train = onehot(y_train_labels)
y_test = onehot(y_test_labels)
# flat may not be needed if input is a tf.dense layer
x_train = flat(x_train)
x_test = flat(x_test)
print(x_train.shape, x_test.shape)